# Database Migration Strategies

Database migration is the process of moving data from one database system to another, whether on-premises to cloud, between cloud providers, or upgrading database versions. This notebook covers essential strategies, tools, and best practices for successful database migrations.

## Migration Approaches Overview

There are three primary approaches to database migration, each with different levels of complexity, cost, and transformation:

| Approach | Complexity | Cost | Downtime | Cloud Benefits |
|----------|------------|------|----------|----------------|
| Lift-and-Shift | Low | Low | Medium | Minimal |
| Re-platform | Medium | Medium | Medium | Moderate |
| Re-architect | High | High | High | Maximum |

## 1. Lift-and-Shift (Rehost)

**Definition**: Moving the database to the cloud with minimal or no changes to the schema, data model, or application code.

### When to Use
- Urgent migration deadlines
- Legacy applications with tight database coupling
- When immediate cloud benefits (DR, scalability) are needed
- Limited budget or resources for refactoring

### Advantages
- ✅ Fastest migration path
- ✅ Lowest risk of introducing bugs
- ✅ Minimal application changes required
- ✅ Easier rollback if issues arise

### Disadvantages
- ❌ May not leverage cloud-native features
- ❌ Technical debt remains
- ❌ Potentially higher operational costs
- ❌ Performance may not be optimized

In [ ]:
# Example: Lift-and-Shift Migration Checklist

lift_shift_checklist = {
    "pre_migration": [
        "Inventory all database objects (tables, views, stored procedures)",
        "Document current performance baselines",
        "Identify database dependencies and connections",
        "Assess storage requirements and IOPS needs",
        "Plan network connectivity (VPN, Direct Connect)"
    ],
    "migration": [
        "Create target database instance with matching specs",
        "Configure security groups and firewall rules",
        "Perform initial data load (full backup/restore)",
        "Set up ongoing replication for sync",
        "Validate data integrity"
    ],
    "post_migration": [
        "Update application connection strings",
        "Verify application functionality",
        "Monitor performance metrics",
        "Decommission source database"
    ]
}

for phase, tasks in lift_shift_checklist.items():
    print(f"\n{phase.upper().replace('_', ' ')}:")
    for i, task in enumerate(tasks, 1):
        print(f"  {i}. {task}")

## 2. Re-platform (Lift-Tinker-and-Shift)

**Definition**: Moving to the cloud with some optimizations to leverage cloud capabilities without changing the core architecture.

### Common Re-platforming Scenarios
- Moving from self-managed MySQL to Amazon RDS MySQL
- Migrating SQL Server to Azure SQL Managed Instance
- Converting Oracle to PostgreSQL-compatible Aurora

### When to Use
- Want managed database benefits (patching, backups, HA)
- Need to reduce operational overhead
- Willing to make minor schema/configuration changes
- Budget for moderate refactoring

### Key Considerations
```
┌─────────────────────────────────────────────────────────────┐
│                    RE-PLATFORM CHANGES                      │
├─────────────────────────────────────────────────────────────┤
│  • Engine version upgrades                                  │
│  • Parameter group/configuration adjustments                │
│  • Authentication mechanism changes (IAM integration)       │
│  • Storage type optimization (SSD, provisioned IOPS)        │
│  • High availability configuration (Multi-AZ, replicas)     │
│  • Backup and retention policy updates                      │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# Example: Re-platform Decision Matrix

replatform_scenarios = [
    {
        "source": "MySQL 5.7 (on-prem)",
        "target": "Amazon RDS MySQL 8.0",
        "changes_needed": [
            "Update deprecated SQL syntax",
            "Review reserved keyword changes",
            "Adjust character set defaults (utf8mb4)",
            "Update authentication plugins"
        ],
        "benefits": ["Managed backups", "Multi-AZ", "Read replicas", "Auto-patching"]
    },
    {
        "source": "SQL Server (on-prem)",
        "target": "Azure SQL Managed Instance",
        "changes_needed": [
            "Remove unsupported features (SQL Agent jobs → Elastic Jobs)",
            "Update linked server configurations",
            "Migrate SSIS packages to Azure Data Factory",
            "Configure Azure AD authentication"
        ],
        "benefits": ["Near 100% compatibility", "Built-in HA", "Automated tuning", "Geo-replication"]
    },
    {
        "source": "PostgreSQL (EC2)",
        "target": "Cloud SQL for PostgreSQL",
        "changes_needed": [
            "Review extension compatibility",
            "Update connection pooling (PgBouncer → Cloud SQL Proxy)",
            "Migrate custom configurations",
            "Set up IAM database authentication"
        ],
        "benefits": ["Fully managed", "Automatic storage", "Point-in-time recovery", "HA with failover"]
    }
]

for scenario in replatform_scenarios:
    print(f"\n{'='*60}")
    print(f"Migration: {scenario['source']} → {scenario['target']}")
    print(f"{'='*60}")
    print("\nChanges Required:")
    for change in scenario['changes_needed']:
        print(f"  • {change}")
    print("\nBenefits Gained:")
    for benefit in scenario['benefits']:
        print(f"  ✓ {benefit}")

## 3. Re-architect (Refactor)

**Definition**: Fundamentally redesigning the database to fully leverage cloud-native services and modern architectures.

### When to Use
- Building for massive scale
- Need to break monolithic databases into microservices
- Want to adopt serverless or event-driven architectures
- Long-term cost optimization is priority

### Re-architecture Patterns

```
MONOLITHIC DATABASE                    CLOUD-NATIVE ARCHITECTURE
┌─────────────────────┐               ┌─────────────────────────────┐
│                     │               │  ┌─────────┐  ┌─────────┐   │
│   Single Oracle     │               │  │ Aurora  │  │DynamoDB │   │
│   Database          │    ────►      │  │ (OLTP)  │  │ (NoSQL) │   │
│                     │               │  └─────────┘  └─────────┘   │
│  - Users            │               │  ┌─────────┐  ┌─────────┐   │
│  - Orders           │               │  │Redshift │  │   S3    │   │
│  - Products         │               │  │ (DWH)   │  │ (Lake)  │   │
│  - Analytics        │               │  └─────────┘  └─────────┘   │
│  - Logs             │               │  ┌─────────┐  ┌─────────┐   │
│                     │               │  │ElastiC. │  │Timestr. │   │
│                     │               │  │ (Search)│  │ (Logs)  │   │
└─────────────────────┘               │  └─────────┘  └─────────┘   │
                                      └─────────────────────────────┘
```

### Database-per-Service Pattern
- Each microservice owns its data
- Choose the right database for each use case
- Use event sourcing for data synchronization
- Implement saga patterns for distributed transactions

In [ ]:
# Example: Re-architect Strategy Planning

rearchitect_analysis = {
    "current_state": {
        "database": "Oracle 12c (Single Instance)",
        "size": "5 TB",
        "tables": 500,
        "daily_transactions": "10M",
        "pain_points": [
            "High licensing costs",
            "Scaling limitations",
            "Complex stored procedures",
            "Mixed workloads (OLTP + Analytics)"
        ]
    },
    "target_architecture": {
        "transactional": {
            "service": "Amazon Aurora PostgreSQL",
            "use_case": "Core business transactions",
            "tables": "~200 (normalized)",
            "features": ["Multi-AZ", "Read replicas", "Serverless v2"]
        },
        "document_store": {
            "service": "Amazon DocumentDB",
            "use_case": "Product catalogs, user profiles",
            "features": ["Flexible schema", "JSON queries", "Auto-scaling"]
        },
        "analytics": {
            "service": "Amazon Redshift Serverless",
            "use_case": "Business intelligence, reporting",
            "features": ["Columnar storage", "ML integration", "Pay-per-query"]
        },
        "cache": {
            "service": "Amazon ElastiCache (Redis)",
            "use_case": "Session management, hot data",
            "features": ["Sub-millisecond latency", "Cluster mode"]
        },
        "search": {
            "service": "Amazon OpenSearch",
            "use_case": "Full-text search, log analytics",
            "features": ["Real-time indexing", "Dashboards"]
        }
    },
    "migration_phases": [
        {"phase": 1, "duration": "3 months", "scope": "Strangler fig pattern - new features on cloud"},
        {"phase": 2, "duration": "6 months", "scope": "Migrate read workloads to purpose-built DBs"},
        {"phase": 3, "duration": "6 months", "scope": "Migrate write workloads with CDC"},
        {"phase": 4, "duration": "3 months", "scope": "Decommission legacy Oracle"}
    ]
}

print("CURRENT STATE ANALYSIS")
print("=" * 50)
current = rearchitect_analysis["current_state"]
print(f"Database: {current['database']}")
print(f"Size: {current['size']} | Tables: {current['tables']} | Daily Txns: {current['daily_transactions']}")
print("\nPain Points:")
for pain in current['pain_points']:
    print(f"  ⚠ {pain}")

print("\n" + "=" * 50)
print("TARGET ARCHITECTURE")
print("=" * 50)
for component, details in rearchitect_analysis["target_architecture"].items():
    print(f"\n{component.upper()}:")
    print(f"  Service: {details['service']}")
    print(f"  Use Case: {details['use_case']}")

print("\n" + "=" * 50)
print("MIGRATION TIMELINE")
print("=" * 50)
for phase in rearchitect_analysis["migration_phases"]:
    print(f"  Phase {phase['phase']} ({phase['duration']}): {phase['scope']}")

## Cloud Database Migration Services

### AWS Database Migration Service (DMS)

AWS DMS enables you to migrate databases to AWS quickly and securely while keeping the source database operational during migration.

#### Key Features
- **Homogeneous migrations**: Oracle to Oracle, MySQL to MySQL
- **Heterogeneous migrations**: Oracle to Aurora PostgreSQL, SQL Server to MySQL
- **Continuous replication**: Keep source and target in sync
- **Schema Conversion Tool (SCT)**: Convert database schemas and code

#### Supported Sources/Targets
| Sources | Targets |
|---------|--------|
| Oracle | Amazon Aurora |
| SQL Server | Amazon RDS |
| MySQL | Amazon Redshift |
| PostgreSQL | Amazon S3 |
| MongoDB | Amazon DynamoDB |
| SAP ASE | Amazon DocumentDB |

In [ ]:
# AWS DMS Task Configuration Example (Conceptual)

aws_dms_config = {
    "replication_instance": {
        "class": "dms.r5.2xlarge",
        "allocated_storage": 100,  # GB
        "multi_az": True,
        "vpc_security_groups": ["sg-migration-001"]
    },
    "source_endpoint": {
        "engine": "oracle",
        "server_name": "oracle-prod.company.com",
        "port": 1521,
        "database_name": "ORCL",
        "ssl_mode": "require",
        "extra_connection_attributes": "useLogMinerReader=Y;useBfile=Y"
    },
    "target_endpoint": {
        "engine": "aurora-postgresql",
        "server_name": "aurora-prod.cluster-xxx.us-east-1.rds.amazonaws.com",
        "port": 5432,
        "database_name": "production",
        "ssl_mode": "require"
    },
    "replication_task": {
        "migration_type": "full-load-and-cdc",  # Full load + ongoing replication
        "table_mappings": {
            "rules": [
                {
                    "rule-type": "selection",
                    "rule-id": "1",
                    "rule-name": "include-all-tables",
                    "object-locator": {
                        "schema-name": "APP_SCHEMA",
                        "table-name": "%"
                    },
                    "rule-action": "include"
                },
                {
                    "rule-type": "transformation",
                    "rule-id": "2",
                    "rule-name": "lowercase-schema",
                    "rule-action": "convert-lowercase",
                    "rule-target": "schema"
                }
            ]
        },
        "task_settings": {
            "TargetMetadata": {
                "BatchApplyEnabled": True,
                "FullLobMode": False,
                "LobChunkSize": 64
            },
            "FullLoadSettings": {
                "TargetTablePrepMode": "DROP_AND_CREATE",
                "MaxFullLoadSubTasks": 8,
                "TransactionConsistencyTimeout": 600
            },
            "Logging": {
                "EnableLogging": True,
                "LogComponents": [
                    {"Id": "SOURCE_UNLOAD", "Severity": "LOGGER_SEVERITY_DEFAULT"},
                    {"Id": "TARGET_LOAD", "Severity": "LOGGER_SEVERITY_DEFAULT"}
                ]
            }
        }
    }
}

print("AWS DMS Configuration Summary")
print("=" * 50)
print(f"\nReplication Instance: {aws_dms_config['replication_instance']['class']}")
print(f"Multi-AZ: {aws_dms_config['replication_instance']['multi_az']}")
print(f"\nSource: {aws_dms_config['source_endpoint']['engine'].upper()}")
print(f"  → {aws_dms_config['source_endpoint']['server_name']}")
print(f"\nTarget: {aws_dms_config['target_endpoint']['engine'].upper()}")
print(f"  → {aws_dms_config['target_endpoint']['server_name']}")
print(f"\nMigration Type: {aws_dms_config['replication_task']['migration_type']}")

### Azure Database Migration Service

Azure DMS provides seamless migrations from multiple database sources to Azure data platforms with minimal downtime.

#### Migration Scenarios

| Scenario | Tool | Mode |
|----------|------|------|
| SQL Server → Azure SQL DB | DMS Online | Near-zero downtime |
| SQL Server → Azure SQL MI | DMS Online | Near-zero downtime |
| MySQL → Azure MySQL | DMS | Offline/Online |
| PostgreSQL → Azure PostgreSQL | DMS | Offline/Online |
| MongoDB → Azure Cosmos DB | DMS | Offline/Online |

#### Key Components
```
┌────────────────────────────────────────────────────────────────┐
│                    AZURE DMS ARCHITECTURE                      │
├────────────────────────────────────────────────────────────────┤
│                                                                │
│  ┌──────────┐    ┌─────────────────┐    ┌──────────────────┐  │
│  │  Source  │───►│  Azure DMS      │───►│  Target Azure    │  │
│  │ Database │    │  (Hybrid Worker)│    │  Database        │  │
│  └──────────┘    └─────────────────┘    └──────────────────┘  │
│       │                  │                       │            │
│       ▼                  ▼                       ▼            │
│  ┌──────────┐    ┌─────────────────┐    ┌──────────────────┐  │
│  │Self-hosted│    │ Data Migration │    │  Azure Monitor   │  │
│  │Integration│    │   Assistant    │    │  (Metrics/Logs)  │  │
│  │  Runtime  │    │    (DMA)       │    │                  │  │
│  └──────────┘    └─────────────────┘    └──────────────────┘  │
│                                                                │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Azure DMS Migration Project Example (Conceptual)

azure_dms_project = {
    "project_name": "SQLServer-to-AzureSQL-Migration",
    "resource_group": "rg-database-migration",
    "location": "eastus",
    "service_tier": "Premium",  # Required for online migrations
    
    "source_connection": {
        "type": "SQL Server",
        "server": "sqlserver-prod.company.local",
        "authentication": "SQL Authentication",
        "encrypt_connection": True,
        "trust_server_certificate": False
    },
    
    "target_connection": {
        "type": "Azure SQL Database",
        "server": "sqlserver-prod.database.windows.net",
        "authentication": "Azure AD",
        "database": "production-db"
    },
    
    "migration_settings": {
        "mode": "Online",  # Continuous sync until cutover
        "backup_file_share": "\\\\fileserver\\sqlbackups",
        "databases": [
            {
                "name": "SalesDB",
                "target_name": "SalesDB",
                "tables_to_migrate": "All"
            },
            {
                "name": "InventoryDB",
                "target_name": "InventoryDB",
                "tables_to_migrate": "All"
            }
        ],
        "validation_options": {
            "data_integrity_validation": True,
            "schema_validation": True,
            "query_analysis_validation": True
        }
    },
    
    "assessment_results": {
        "compatibility_level": "150",
        "blocking_issues": 0,
        "warnings": 3,
        "estimated_migration_time": "4 hours"
    }
}

print("Azure DMS Migration Project")
print("=" * 50)
print(f"Project: {azure_dms_project['project_name']}")
print(f"Mode: {azure_dms_project['migration_settings']['mode']}")
print(f"\nSource: {azure_dms_project['source_connection']['type']}")
print(f"  Server: {azure_dms_project['source_connection']['server']}")
print(f"\nTarget: {azure_dms_project['target_connection']['type']}")
print(f"  Server: {azure_dms_project['target_connection']['server']}")
print(f"\nDatabases to Migrate:")
for db in azure_dms_project['migration_settings']['databases']:
    print(f"  • {db['name']} → {db['target_name']}")
print(f"\nAssessment Results:")
print(f"  Blocking Issues: {azure_dms_project['assessment_results']['blocking_issues']}")
print(f"  Warnings: {azure_dms_project['assessment_results']['warnings']}")
print(f"  Estimated Time: {azure_dms_project['assessment_results']['estimated_migration_time']}")

### Google Cloud Database Migration Service

GCP DMS provides serverless, minimal-downtime migrations to Cloud SQL and AlloyDB.

#### Supported Migration Paths
- MySQL → Cloud SQL for MySQL
- PostgreSQL → Cloud SQL for PostgreSQL / AlloyDB
- SQL Server → Cloud SQL for SQL Server
- Oracle → Bare Metal Solution / PostgreSQL (via heterogeneous migration)

#### Key Features
- **Serverless**: No infrastructure to manage
- **Continuous migration**: Real-time replication
- **Minimal downtime**: Seconds of downtime during cutover
- **Built-in validation**: Data validation before cutover

In [ ]:
# GCP DMS Migration Job Example (Conceptual)

gcp_dms_job = {
    "migration_job_id": "mysql-to-cloudsql-prod",
    "display_name": "Production MySQL Migration",
    "type": "CONTINUOUS",  # CDC replication
    
    "source": {
        "connection_profile": "mysql-source-profile",
        "engine": "MYSQL",
        "version": "8.0",
        "host": "mysql-prod.company.com",
        "port": 3306,
        "ssl_config": {
            "type": "SERVER_ONLY",
            "ca_certificate": "projects/my-project/locations/us-central1/certificates/mysql-ca"
        },
        "connectivity": "VPC_PEERING"  # or REVERSE_SSH_TUNNEL
    },
    
    "destination": {
        "connection_profile": "cloudsql-dest-profile",
        "cloud_sql_id": "projects/my-project/instances/mysql-prod-cloudsql",
        "instance_tier": "db-custom-8-32768",
        "storage_size_gb": 500,
        "availability_type": "REGIONAL"  # High availability
    },
    
    "migration_settings": {
        "dump_parallelism": 4,
        "dump_path": "gs://migration-bucket/mysql-dump",
        "filter": {
            "databases": ["app_db", "analytics_db"],
            "excluded_tables": ["app_db.temp_*", "app_db.logs_*"]
        }
    },
    
    "phases": [
        {"phase": "FULL_DUMP", "status": "COMPLETED", "duration": "45 min"},
        {"phase": "CDC", "status": "RUNNING", "lag": "2 seconds"},
        {"phase": "PROMOTE_IN_PROGRESS", "status": "PENDING", "duration": None}
    ]
}

print("GCP Database Migration Service Job")
print("=" * 50)
print(f"Job ID: {gcp_dms_job['migration_job_id']}")
print(f"Type: {gcp_dms_job['type']}")
print(f"\nSource: {gcp_dms_job['source']['engine']} {gcp_dms_job['source']['version']}")
print(f"  Host: {gcp_dms_job['source']['host']}")
print(f"  Connectivity: {gcp_dms_job['source']['connectivity']}")
print(f"\nDestination: Cloud SQL")
print(f"  Instance: {gcp_dms_job['destination']['cloud_sql_id'].split('/')[-1]}")
print(f"  Tier: {gcp_dms_job['destination']['instance_tier']}")
print(f"\nMigration Phases:")
for phase in gcp_dms_job['phases']:
    status_icon = "✓" if phase['status'] == 'COMPLETED' else "⟳" if phase['status'] == 'RUNNING' else "○"
    duration_str = f" ({phase['duration']})" if phase['duration'] else ""
    lag_str = f" - Lag: {phase['lag']}" if 'lag' in phase and phase['status'] == 'RUNNING' else ""
    print(f"  {status_icon} {phase['phase']}: {phase['status']}{duration_str}{lag_str}")

## Zero-Downtime Migration Techniques

Achieving zero or near-zero downtime during database migration requires careful planning and the right combination of techniques.

### Core Techniques

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    ZERO-DOWNTIME MIGRATION FLOW                         │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  Phase 1: Initial Load          Phase 2: CDC Sync                       │
│  ┌──────────┐                   ┌──────────┐                            │
│  │  Source  │ ────────────────► │  Target  │                            │
│  │    DB    │   Full Snapshot   │    DB    │                            │
│  └──────────┘                   └──────────┘                            │
│       │                              ▲                                  │
│       │     ┌──────────────┐         │                                  │
│       └────►│  Change Data │─────────┘                                  │
│             │   Capture    │  Continuous                                │
│             └──────────────┘  Replication                               │
│                                                                         │
│  Phase 3: Validation            Phase 4: Cutover                        │
│  ┌────────────────────┐        ┌─────────────────┐                      │
│  │ Row counts match   │        │ Stop writes     │                      │
│  │ Checksums verify   │   ──►  │ Final sync      │                      │
│  │ Sample data compare│        │ Switch apps     │                      │
│  └────────────────────┘        └─────────────────┘                      │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Zero-Downtime Migration Strategy Implementation

zero_downtime_strategy = {
    "technique": "Blue-Green with CDC",
    
    "phases": {
        "1_preparation": {
            "duration": "1-2 weeks",
            "tasks": [
                "Set up target database infrastructure",
                "Configure network connectivity (VPN/peering)",
                "Create replication user with minimal privileges",
                "Enable binary logging (MySQL) or logical replication (PostgreSQL)",
                "Test failover procedures in staging"
            ]
        },
        "2_initial_load": {
            "duration": "Hours to days (size dependent)",
            "tasks": [
                "Take consistent snapshot of source database",
                "Transfer snapshot to target (parallel load for speed)",
                "Note the binlog/LSN position at snapshot time",
                "Restore snapshot to target database",
                "Build indexes after data load (faster than during)"
            ],
            "tips": [
                "Use parallel dump/restore for large databases",
                "Disable foreign key checks during load",
                "Consider using native cloud import tools"
            ]
        },
        "3_cdc_replication": {
            "duration": "Continuous until cutover",
            "tasks": [
                "Start CDC from recorded binlog/LSN position",
                "Monitor replication lag continuously",
                "Handle schema changes carefully (DDL replication)",
                "Ensure replication catches up before cutover"
            ],
            "monitoring_metrics": [
                "Replication lag (seconds behind master)",
                "Transactions per second replicated",
                "Error rate and skipped transactions"
            ]
        },
        "4_validation": {
            "duration": "1-2 days",
            "tasks": [
                "Compare row counts for all tables",
                "Verify checksum of sample data",
                "Run application test suite against target",
                "Validate stored procedures and functions",
                "Performance benchmark critical queries"
            ]
        },
        "5_cutover": {
            "duration": "Seconds to minutes",
            "steps": [
                "1. Announce maintenance window (even if minimal)",
                "2. Stop application writes to source",
                "3. Wait for replication to fully catch up (lag = 0)",
                "4. Verify final row counts match",
                "5. Update application connection strings",
                "6. Start applications pointing to new database",
                "7. Monitor for errors",
                "8. Keep source database read-only for rollback (24-48h)"
            ]
        }
    },
    
    "rollback_plan": {
        "trigger_conditions": [
            "Data integrity issues detected",
            "Performance degradation > 50%",
            "Critical application errors",
            "Replication lag cannot be recovered"
        ],
        "rollback_steps": [
            "Stop applications",
            "Revert connection strings to source",
            "Restart applications",
            "Sync any target changes back to source (if needed)"
        ]
    }
}

print("ZERO-DOWNTIME MIGRATION STRATEGY")
print("=" * 60)
print(f"Technique: {zero_downtime_strategy['technique']}")

for phase_name, phase_details in zero_downtime_strategy['phases'].items():
    print(f"\n{'─' * 60}")
    print(f"Phase: {phase_name.replace('_', ' ').upper()}")
    print(f"Duration: {phase_details['duration']}")
    
    if 'tasks' in phase_details:
        print("Tasks:")
        for task in phase_details['tasks']:
            print(f"  □ {task}")
    
    if 'steps' in phase_details:
        print("Steps:")
        for step in phase_details['steps']:
            print(f"  {step}")

### Dual-Write Pattern

An alternative approach where the application writes to both databases simultaneously during migration.

```
┌─────────────────────────────────────────────────────────────┐
│                     DUAL-WRITE PATTERN                      │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│                    ┌─────────────┐                          │
│                    │ Application │                          │
│                    └──────┬──────┘                          │
│                           │                                 │
│                    ┌──────▼──────┐                          │
│                    │   Write     │                          │
│                    │   Router    │                          │
│                    └──────┬──────┘                          │
│                  ┌────────┴────────┐                        │
│                  │                 │                        │
│           ┌──────▼──────┐   ┌──────▼──────┐                 │
│           │   Source    │   │   Target    │                 │
│           │  (Primary)  │   │ (Secondary) │                 │
│           └─────────────┘   └─────────────┘                 │
│                                                             │
│  Phase 1: Source primary, Target secondary                  │
│  Phase 2: Both active, source primary                       │
│  Phase 3: Both active, target primary                       │
│  Phase 4: Target primary only                               │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

**Caution**: Dual-write adds complexity and potential consistency issues. Use CDC-based approaches when possible.

## Data Validation and Cutover Strategies

### Validation Framework

Data validation ensures that the migrated data is complete, accurate, and consistent with the source.

In [ ]:
# Comprehensive Data Validation Framework

class MigrationValidator:
    """Framework for validating database migrations."""
    
    def __init__(self, source_db, target_db):
        self.source = source_db
        self.target = target_db
        self.validation_results = []
    
    def validate_row_counts(self, tables):
        """Compare row counts between source and target."""
        results = []
        for table in tables:
            # Simulated counts for demonstration
            source_count = 1000000  # Would query source
            target_count = 1000000  # Would query target
            
            results.append({
                'table': table,
                'source_count': source_count,
                'target_count': target_count,
                'match': source_count == target_count,
                'difference': source_count - target_count
            })
        return results
    
    def validate_checksums(self, table, key_columns, sample_size=1000):
        """Compare checksums of sample data."""
        # Query pattern for checksum comparison
        checksum_query = f"""
        SELECT 
            MD5(CONCAT_WS('|', {', '.join(key_columns)})) as row_hash,
            {', '.join(key_columns)}
        FROM {table}
        ORDER BY RANDOM()
        LIMIT {sample_size}
        """
        return {"query": checksum_query, "sample_size": sample_size}
    
    def validate_constraints(self):
        """Verify all constraints are properly migrated."""
        constraint_checks = {
            'primary_keys': "Check all PKs exist and are enforced",
            'foreign_keys': "Verify FK relationships and cascades",
            'unique_constraints': "Confirm unique constraints active",
            'check_constraints': "Validate check constraints migrated",
            'not_null': "Ensure NOT NULL constraints preserved"
        }
        return constraint_checks
    
    def validate_sequences(self):
        """Ensure sequences are synced and won't conflict."""
        return {
            'check': "Target sequence values > max source values",
            'buffer': "Add 10000 buffer to sequence values",
            'verification': "Test insert with auto-increment"
        }

# Validation Checklist
validation_checklist = {
    "pre_cutover_validation": [
        {
            "check": "Row Count Verification",
            "description": "All tables have matching row counts",
            "tolerance": "0% difference",
            "blocking": True
        },
        {
            "check": "Schema Comparison",
            "description": "All objects (tables, views, procedures) exist",
            "tolerance": "100% match",
            "blocking": True
        },
        {
            "check": "Primary Key Integrity",
            "description": "No duplicate PKs, all PKs migrated",
            "tolerance": "0 violations",
            "blocking": True
        },
        {
            "check": "Sample Data Checksum",
            "description": "1000 random rows match source exactly",
            "tolerance": "100% match",
            "blocking": True
        },
        {
            "check": "Replication Lag",
            "description": "CDC is caught up with source",
            "tolerance": "< 5 seconds lag",
            "blocking": True
        },
        {
            "check": "Critical Query Performance",
            "description": "Top 10 queries perform within SLA",
            "tolerance": "< 20% slower than source",
            "blocking": False
        },
        {
            "check": "Application Smoke Tests",
            "description": "Core application functions work",
            "tolerance": "All tests pass",
            "blocking": True
        }
    ],
    
    "post_cutover_validation": [
        "Monitor error rates in application logs",
        "Track database connection failures",
        "Verify batch job completions",
        "Check replication to read replicas",
        "Validate backup jobs running"
    ]
}

print("DATA VALIDATION CHECKLIST")
print("=" * 60)
print("\nPRE-CUTOVER VALIDATION")
print("-" * 60)
for item in validation_checklist['pre_cutover_validation']:
    blocking = "🚫 BLOCKING" if item['blocking'] else "⚠️ WARNING"
    print(f"\n{blocking}: {item['check']}")
    print(f"  Description: {item['description']}")
    print(f"  Tolerance: {item['tolerance']}")

print("\n" + "-" * 60)
print("POST-CUTOVER VALIDATION")
print("-" * 60)
for item in validation_checklist['post_cutover_validation']:
    print(f"  □ {item}")

### Cutover Strategies

The cutover is the critical moment when traffic switches from source to target database.

In [ ]:
# Cutover Strategy Comparison

cutover_strategies = {
    "big_bang": {
        "description": "All applications switch to new database at once",
        "downtime": "Minutes to hours",
        "risk": "High",
        "complexity": "Low",
        "best_for": "Small databases, simple applications",
        "steps": [
            "Schedule maintenance window",
            "Stop all applications",
            "Final data sync",
            "Validate data",
            "Update all connection strings",
            "Start all applications",
            "Verify functionality"
        ]
    },
    
    "phased_rollout": {
        "description": "Migrate applications in groups/phases",
        "downtime": "Per-application, minimal",
        "risk": "Medium",
        "complexity": "Medium",
        "best_for": "Multiple applications, different criticality",
        "steps": [
            "Categorize apps by criticality",
            "Migrate non-critical apps first",
            "Monitor and validate each phase",
            "Progressively migrate critical apps",
            "Keep dual-write for rollback capability"
        ]
    },
    
    "canary_release": {
        "description": "Route small percentage of traffic to new DB",
        "downtime": "Zero",
        "risk": "Low",
        "complexity": "High",
        "best_for": "High-traffic, critical systems",
        "steps": [
            "Set up traffic routing (load balancer/proxy)",
            "Route 1% of read traffic to target",
            "Monitor errors and latency",
            "Gradually increase to 10%, 25%, 50%",
            "Switch writes when confident",
            "Complete migration to 100%"
        ],
        "rollback": "Instantly route traffic back to source"
    },
    
    "blue_green": {
        "description": "Two identical environments, instant switch",
        "downtime": "Seconds",
        "risk": "Low",
        "complexity": "High",
        "best_for": "Mission-critical systems, strict SLAs",
        "steps": [
            "Maintain both environments in sync",
            "Validate green (target) environment",
            "DNS/Load balancer switch to green",
            "Monitor for issues",
            "Keep blue (source) for rollback",
            "Decommission blue after stability period"
        ],
        "rollback": "Switch DNS/LB back to blue environment"
    }
}

print("CUTOVER STRATEGY COMPARISON")
print("=" * 70)

# Summary table
print(f"\n{'Strategy':<20} {'Downtime':<15} {'Risk':<10} {'Complexity':<12}")
print("-" * 70)
for name, details in cutover_strategies.items():
    print(f"{name.replace('_', ' ').title():<20} {details['downtime']:<15} {details['risk']:<10} {details['complexity']:<12}")

# Detailed breakdown
print("\n" + "=" * 70)
for name, details in cutover_strategies.items():
    print(f"\n{name.replace('_', ' ').upper()}")
    print("-" * 40)
    print(f"Description: {details['description']}")
    print(f"Best For: {details['best_for']}")
    print("\nSteps:")
    for i, step in enumerate(details['steps'], 1):
        print(f"  {i}. {step}")
    if 'rollback' in details:
        print(f"\nRollback: {details['rollback']}")

### Cutover Runbook Template

In [ ]:
# Cutover Runbook Template

cutover_runbook = {
    "migration_name": "Oracle to Aurora PostgreSQL Migration",
    "scheduled_date": "2026-02-15 02:00 UTC",
    "estimated_duration": "45 minutes",
    "max_rollback_time": "15 minutes",
    
    "team_contacts": {
        "migration_lead": "DBA Team Lead - (555) 123-4567",
        "application_owner": "App Team Lead - (555) 234-5678",
        "infrastructure": "Infra On-Call - (555) 345-6789",
        "escalation": "VP Engineering - (555) 456-7890"
    },
    
    "pre_cutover_checklist": [
        {"task": "Confirm maintenance window communicated", "owner": "PM", "time": "T-24h"},
        {"task": "Validate all CDC replication healthy", "owner": "DBA", "time": "T-4h"},
        {"task": "Run pre-cutover validation suite", "owner": "DBA", "time": "T-2h"},
        {"task": "Confirm rollback procedure tested", "owner": "DBA", "time": "T-2h"},
        {"task": "All team members available on bridge", "owner": "All", "time": "T-30m"},
        {"task": "Monitoring dashboards ready", "owner": "SRE", "time": "T-30m"}
    ],
    
    "cutover_steps": [
        {"step": 1, "action": "Announce cutover start on bridge", "duration": "1 min", "rollback": "N/A"},
        {"step": 2, "action": "Enable application maintenance mode", "duration": "2 min", "rollback": "Disable maintenance mode"},
        {"step": 3, "action": "Stop application write connections", "duration": "2 min", "rollback": "Re-enable connections"},
        {"step": 4, "action": "Wait for replication lag = 0", "duration": "5 min", "rollback": "N/A"},
        {"step": 5, "action": "Verify final row counts match", "duration": "5 min", "rollback": "Investigate discrepancy"},
        {"step": 6, "action": "Stop CDC replication", "duration": "1 min", "rollback": "Resume replication"},
        {"step": 7, "action": "Update DNS/connection strings", "duration": "5 min", "rollback": "Revert DNS changes"},
        {"step": 8, "action": "Disable maintenance mode", "duration": "2 min", "rollback": "Enable maintenance mode"},
        {"step": 9, "action": "Verify application connectivity", "duration": "5 min", "rollback": "Check connection settings"},
        {"step": 10, "action": "Run smoke tests", "duration": "10 min", "rollback": "Initiate rollback"},
        {"step": 11, "action": "Monitor for 15 min stability", "duration": "15 min", "rollback": "Initiate rollback"},
        {"step": 12, "action": "Announce cutover complete", "duration": "1 min", "rollback": "N/A"}
    ],
    
    "rollback_triggers": [
        "Data integrity validation fails",
        "Application error rate > 5%",
        "P99 latency > 2x baseline",
        "Critical functionality not working",
        "Team consensus to rollback"
    ],
    
    "success_criteria": [
        "All applications connected to new database",
        "Error rates within normal range",
        "Response times within SLA",
        "All batch jobs completed successfully",
        "No data integrity issues reported"
    ]
}

print("DATABASE MIGRATION CUTOVER RUNBOOK")
print("=" * 70)
print(f"\nMigration: {cutover_runbook['migration_name']}")
print(f"Scheduled: {cutover_runbook['scheduled_date']}")
print(f"Duration: {cutover_runbook['estimated_duration']}")
print(f"Max Rollback Window: {cutover_runbook['max_rollback_time']}")

print("\nTEAM CONTACTS")
print("-" * 40)
for role, contact in cutover_runbook['team_contacts'].items():
    print(f"  {role.replace('_', ' ').title()}: {contact}")

print("\nCUTOVER STEPS")
print("-" * 70)
print(f"{'Step':<6} {'Action':<40} {'Duration':<10}")
print("-" * 70)
for step in cutover_runbook['cutover_steps']:
    print(f"{step['step']:<6} {step['action']:<40} {step['duration']:<10}")

print("\nROLLBACK TRIGGERS")
print("-" * 40)
for trigger in cutover_runbook['rollback_triggers']:
    print(f"  🚨 {trigger}")

print("\nSUCCESS CRITERIA")
print("-" * 40)
for criteria in cutover_runbook['success_criteria']:
    print(f"  ✓ {criteria}")

## Takeaways

### Key Principles for Successful Database Migration

| Principle | Description |
|-----------|-------------|
| **Assess First** | Thoroughly analyze source database, dependencies, and workload patterns before choosing a strategy |
| **Choose the Right Approach** | Match lift-and-shift, re-platform, or re-architect to your timeline, budget, and goals |
| **Use Managed Tools** | Leverage cloud provider DMS tools (AWS, Azure, GCP) for reliable, battle-tested migrations |
| **Prioritize Zero-Downtime** | Use CDC-based replication for minimal business disruption |
| **Validate Extensively** | Never skip data validation—row counts, checksums, and application testing |
| **Plan Rollback** | Always have a tested rollback plan with clear triggers |

### Migration Strategy Decision Tree

```
                        START
                          │
                          ▼
              ┌───────────────────────┐
              │ Urgent deadline or    │───Yes──► LIFT-AND-SHIFT
              │ minimal budget?       │
              └───────────┬───────────┘
                          │ No
                          ▼
              ┌───────────────────────┐
              │ Need managed DB       │───Yes──► RE-PLATFORM
              │ without major changes?│
              └───────────┬───────────┘
                          │ No
                          ▼
              ┌───────────────────────┐
              │ Building for scale or │───Yes──► RE-ARCHITECT
              │ cloud-native features?│
              └───────────┬───────────┘
                          │ No
                          ▼
                    RE-PLATFORM
                   (Safe default)
```

### Common Pitfalls to Avoid

1. **Underestimating data volume** - Test with production-scale data
2. **Ignoring network latency** - Set up proper connectivity early
3. **Skipping application testing** - Run full test suites against target
4. **No rollback plan** - Always have a tested escape route
5. **Rushing cutover** - Wait for zero replication lag
6. **Forgetting about sequences** - Ensure auto-increment values won't collide
7. **Not monitoring after cutover** - Watch closely for 24-48 hours

### Recommended Resources

- [AWS Database Migration Service](https://docs.aws.amazon.com/dms/)
- [Azure Database Migration Guide](https://learn.microsoft.com/en-us/azure/dms/)
- [Google Cloud Database Migration Service](https://cloud.google.com/database-migration)
- [Martin Fowler's Strangler Fig Pattern](https://martinfowler.com/bliki/StranglerFigApplication.html)
- [AWS Schema Conversion Tool](https://docs.aws.amazon.com/SchemaConversionTool/)